In [2]:
from PyPDF2 import PdfReader
import re,os
import pandas as pd
import numpy as np

In [3]:
#This is not good but works


In [4]:
for file in os.listdir('GabaritosLaranjaPDF'):
    enem = PdfReader('GabaritosLaranjaPDF/'+file)
    try:
        text = enem.pages[0].extract_text()
        text = re.sub('\xa0',' ',text)
        text = re.sub(' +',' ',text)
        is_english = 'inglês' in text.lower()
        gab = [i.split(' ') for i in re.findall(r'[\d]+ [A-E](?: [A-E])?(?!nulad|NULAD)',text)]
        #treat prova de espanhol:
        new_gab = []
        for answ in gab:
            if len(answ)>2:
                new_gab.append(np.array([answ[0],answ[2],'spanish']))
                new_gab.append(np.array([answ[0],answ[1],'english']))
            else:
                new_gab.append(np.array(answ+[None]))
        gab = new_gab
        gab = np.stack(gab)
        df = pd.DataFrame({'question':gab[:,0],'answer':gab[:,1],'test':gab[:,2]})
        df['question'] = pd.to_numeric(df['question'])
        df=df.sort_values('question').reset_index(drop=True)
        min_quest = 1 if df['question'].min() < 90 else 91
        max_quest = min_quest+90
        
    except Exception as e:
        print(e)
    if (len(df)!=90 and len(df)!=95) and not 'anulad' in text.lower():
        raise Exception('Enem has 90 questions')
    elif (len(df)!=90 or len(df)!=95):
        for question in range(min_quest,max_quest):
            if question not in df['question'].values:
                df.loc[len(df)] = [question,'anulada',None]
        if is_english:
            for question in range(min_quest,min_quest+5):
                if len(df[df['question']==question]) < 2:
                    if 'english' not in df[df['question']==question]['question'].tolist():
                        df.loc[len(df)] = [question,'anulada','english']
                    if 'spanish' not in df[df['question']==question]['question'].tolist():
                        df.loc[len(df)] = [question,'anulada','spanish']
    df.sort_values('question').to_csv('GabaritosCsv/'+file.strip('.pdf')+'.csv',index=False)

In [5]:
df

,question,answer,test
0,91,E,english
1,91,C,spanish
2,92,C,spanish
3,92,D,english
4,93,B,spanish
...,...,...,...
90,176,E,None
91,177,A,None
92,178,C,None
93,179,B,None


In [21]:
file

'gabarito_dia1_2009.pdf'

In [22]:
text

'Enem 2009 CADERNOS DO 1° DIA - 05/12/2009\nCaderno 1 Caderno 2 Caderno 3 Caderno 4\nAzul Gab_Azul Amarelo Gab_Amarelo Branco Gab_Branco Rosa Gab_Rosa\n1 C 1 C 1 D 1 B\n2 D 2 A 2 A 2 E\n3 A 3 D 3 C 3 D\n4 B 4 B 4 E 4 D\n5 E 5 E 5 B 5 C\n6 D 6 D 6 D 6 A\n7 D 7 D 7 D 7 D\n8 B 8 B 8 C 8 B\n9 D 9 D 9 D 9 D\n10 C 10 C 10 C 10 C\n11 C 11 C 11 B 11 C\n12 A 12 A 12 D 12 A\n13 D 13 D 13 A 13 D\n14 E 14 E 14 E 14 E\n15 B 15 B 15 D 15 D\n16 D 16 D 16 B 16 B\n17 E 17 E 17 E 17 E\n18 E 18 E 18 E 18 E\n19 D 19 D 19 D 19 D\n20 E 20 E 20 C 20 E\n21 C 21 D 21 E 21 D\n22 D 22 C 22 D 22 C\n23 E 23 E 23 E 23 E\n24 A 24 A 24 A 24 A\n25 B 25 D 25 D 25 B\n26 D 26 B 26 B 26 B\n27 D 27 D 27 D 27 D\n28 B 28 B 28 B 28 D\n29 D 29 D 29 D 29 D\n30 D 30 B 30 E 30 B\n31 E 31 D 31 D 31 D\n32 B 32 E 32 B 32 E\n33 C 33 C 33 C 33 C\n34 C 34 C 34 C 34 C\n35 A 35 A 35 E 35 A\n36 E 36 E 36 A 36 E\n37 D 37 D 37 D 37 D\n38 D 38 D 38 D 38 D\n39 B 39 B 39 B 39 B\n40 D 40 A 40 D 40 A\n41 A 41 D 41 A 41 D\n42 E 42 E 42 A 42 A\n43